In [1]:
import pandas as pd
import pysam
import numpy as np
import sys
from collections import defaultdict
import time
start_time = time.time()


bamfile='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/veryefficient/CD4allrange_NR_1000000_insilmix41_sorted'
sm='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/veryefficient/BL14_all_matrixCin_nr0.4_imputed_rowmean.txt_bg_intesectedwith_CD4DMRofBL14atleast.2SM_g1_CD4_3_g2_others_33_withheader.txt_othermean.txt'
mapping_quality=40


In [2]:
smdf=pd.read_csv(sm,sep='\t',index_col=['chrom','start'])

smdf.head()    

CD4-others  CD8-others  nB-others  NK-others  Mn-others  \
chrom start                                                               
chr1  32387132   -0.707256   -0.392077   0.130231   0.104744   0.105103   
      32387376   -0.716974   -0.172051   0.165744   0.095385   0.223179   
      32387403   -0.783192    0.005833   0.101679   0.091269   0.146551   
      32387409   -0.764192    0.083705   0.105244   0.106321   0.097346   
      32387417   -0.295179    0.017846   0.042615   0.016769   0.063795   

                mNeu-others  m8-others  DC-others  Eo-others  Tr-others  \
chrom start                                                               
chr1  32387132     0.101872   0.076385   0.118385   0.094154  -0.001692   
      32387376     0.072410   0.040103   0.145641  -0.019128  -0.107436   
      32387403     0.096654  -0.000449   0.146551   0.113167  -0.093603   
      32387409     0.070423   0.105603   0.116372   0.116372  -0.145321   
      32387417    -0.030256   0.063795   0.043692   0.027718  -0.011590   

                m4-others  Mg-others  Er-others  mB-others  
chrom start                                                 
chr1  32387132   0.052692   0.138846   0.054846   0.123769  
      32387376  -0.061128   0.113872   0.109564   0.110821  
      32387403  -0.164141   0.077090   0.132551   0.130038  
      32387409  -0.155013   0.093218   0.103449   0.066474  
      32387417  -0.032590   0.023949   0.040103   0.029333

In [3]:
scorecolumns=smdf.columns
scoredict=defaultdict(list)

In [4]:
def populate_score_matrix(Readname,ReadAlignedPairs,Readbases,Readbase_qualities,direction,readChrom):
    prev_base='N'
    prevRefpos=-99999
    prevBasepos=-9999
    for t in ReadAlignedPairs:
        currentbase=t[2]
        if currentbase==None:
            continue
        currentbase=currentbase.upper()
        currentRefpos=t[1]
        currentBasepos=t[0]
   
        if (currentbase=='G' and prev_base=='C') and (currentRefpos==prevRefpos+1):
            corressSMindex=(readChrom,prevRefpos)
           
            try:
                smrow=(smdf.loc[corressSMindex]).to_numpy()
                
              
                
               
                
            except:
               
                continue
            
            
            accpeted=0
            
            if direction==0:
                
                if prevBasepos==None:
                    continue
                    
              
                readbase = (Readbases[prevBasepos]).upper()
                
                if readbase=='T':
                    accpeted=1
                elif readbase==prev_base:
                    accpeted=-1
                
                
            elif direction==1:
                if currentBasepos==None:
                    continue
                readbase = (Readbases[currentBasepos]).upper()
                
                if readbase=='A':
                    accpeted=1
                elif readbase==currentbase:
                    accpeted=-1
               
                
            
            
            
          
            
            if accpeted!=0 :
            
            
                if len(scoredict[Readname])==0:
                    scoredict[Readname]=list(accpeted*smrow)
                else:
                    scoredict[Readname]=list(np.array(scoredict[Readname])+(accpeted*smrow))
            
            

           
            
            
        prev_base=currentbase
        prevRefpos=currentRefpos
        prevBasepos=currentBasepos
        
    

In [5]:
bam=pysam.AlignmentFile(bamfile, 'rb')

for read in bam.fetch():
    if read.mapping_quality < mapping_quality:
        continue
    if read.is_duplicate == True:
        continue
    
    offset = 0

    

    if read.has_tag('YD'):
        yd = read.get_tag("YD")
        if yd == "r":
            offset = 1

    elif read.has_tag('XB'):
        xb=read.get_tag("XB")

        if xb=='G':
            offset=1
        elif xb=='C':
            pass
        else:
            print("unknown xb value. Exiting....")
            print(read.query_name)

            sys.exit(1)

    else:
        print("no tag found. Exiting")
        print(read.query_name)
        sys.exit(1)
        
        
    
    
    try:
        alignedpairs=read.get_aligned_pairs(with_seq=True)
        
        
        
    except:
        continue
    
    chrom = read.reference_name
    readbase_qualities=read.query_qualities
    readbases=read.query_sequence

    populate_score_matrix(read.query_name,alignedpairs,readbases,readbase_qualities,offset,chrom)
    

In [6]:
scoredf=pd.DataFrame.from_dict(scoredict, orient='index',columns=scorecolumns)
scoredf.head()

,CD4-others,CD8-others,nB-others,NK-others,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,Tr-others,m4-others,Mg-others,Er-others,mB-others
Tr.HWI-ST537:240:C6B3NACXX:4:1210:3799:3014,0.707256,0.392077,-0.130231,-0.104744,-0.105103,-0.101872,-0.076385,-0.118385,-0.094154,0.001692,-0.052692,-0.138846,-0.054846,-0.123769
Tr.HWI-ST700660:269:C4UNFACXX:3:1108:14410:48102,0.707256,0.392077,-0.130231,-0.104744,-0.105103,-0.101872,-0.076385,-0.118385,-0.094154,0.001692,-0.052692,-0.138846,-0.054846,-0.123769
mNeu.HWI-ST731_20:4:2316:17867:33158#0,1.414513,0.784154,-0.260462,-0.209487,-0.210205,-0.203744,-0.152769,-0.236769,-0.188308,0.003385,-0.105385,-0.277692,-0.109692,-0.247538
nB.HWI-ST858_121:7:2312:16040:85640#7,-0.707256,-0.392077,0.130231,0.104744,0.105103,0.101872,0.076385,0.118385,0.094154,-0.001692,0.052692,0.138846,0.054846,0.123769
nB.HWI-ST731_75:4:1211:2471:95882#19,0.707256,0.392077,-0.130231,-0.104744,-0.105103,-0.101872,-0.076385,-0.118385,-0.094154,0.001692,-0.052692,-0.138846,-0.054846,-0.123769


In [8]:
scoredf.to_csv(bamfile+"_scored.txt",sep="\t")

In [7]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

842.9273080825806
